In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import torch
import pandas as pd
import warnings
from tensorflow.keras.models import Model
from keras.utils import to_categorical
from transformers import BertTokenizer, BertModel

In [3]:
warnings.simplefilter(action='ignore', category=FutureWarning)

In [4]:
OKATO_REGIONS = '''01	Алтайский край
03	Краснодарский край
04	Красноярский край
05	Приморский край
07	Ставропольский край
08	Хабаровский край
10	Амурская область
11	Архангельская область
12	Астраханская область
14	Белгородская область
15	Брянская область
17	Владимирская область
18	Волгоградская область
19	Вологодская область
20	Воронежская область
22	Нижегородская область
24	Ивановская область
25	Иркутская область
26	Республика Ингушетия
27	Калининградская область
28	Тверская область
29	Калужская область
30	Камчатский край
32	Кемеровская область - Кузбасс
33	Кировская область
34	Костромская область
35	Республика Крым
36	Самарская область
37	Курганская область
38	Курская область
40	Город Санкт-Петербург город федерального значения	
41	Ленинградская область
42	Липецкая область
44	Магаданская область
45	Город Москва столица Российской Федерации город федерального значения	
46	Московская область
47	Мурманская область
49	Новгородская область
50	Новосибирская область
52	Омская область
53	Оренбургская область
54	Орловская область
56	Пензенская область
57	Пермский край
58	Псковская область
60	Ростовская область
61	Рязанская область
63	Саратовская область
64	Сахалинская область
65	Свердловская область
66	Смоленская область
67	Город федерального значения Севастополь	
68	Тамбовская область
69	Томская область
70	Тульская область
71	Тюменская область
73	Ульяновская область
75	Челябинская область
76	Забайкальский край
77	Чукотский автономный округ
78	Ярославская область
79	Республика Адыгея (Адыгея)
80	Республика Башкортостан
81	Республика Бурятия
82	Республика Дагестан
83	Кабардино-Балкарская Республика
84	Республика Алтай
85	Республика Калмыкия
86	Республика Карелия
87	Республика Коми
88	Республика Марий Эл
89	Республика Мордовия
90	Республика Северная Осетия-Алания
91	Карачаево-Черкесская Республика
92	Республика Татарстан (Татарстан)
93	Республика Тыва
94	Удмуртская Республика
95	Республика Хакасия
96	Чеченская Республика
97	Чувашская Республика - Чувашия
98	Республика Саха (Якутия)
99	Еврейская автономная область'''.split('\n')

In [5]:
regions = {}
for i in OKATO_REGIONS:
    row = i.split('\t')
    regions[row[0]] = row[1]

In [35]:
mappings = {
    'Полное имя': ['full_name', 'name', 'FIO'],
    'Пол': ['gender', 'sex', 'm_or_f'],
    'Дата рождения': ['birth_date', 'date_of_birth', 'dob', 'birthdate', 'bdate'],
    'Паспорт': ['passport_number', 'passport_id', 'passport_series_and_number', 'passport', 'pass', 'pass_id', 'pass_num', 'pass_number'],
    'Место и адрес жительства': ['address', 'residence_address', 'living_address', 'living_place'],
    'Номер телефона': ['phone_number', 'phone', 'mobile_phone', 'pnumber', 'p_number', 'phone_n', 'number'],
    'Сведения о доходах': ['income_info', 'salary_info', 'earnings_info', 'income', 'salary', 'earnings'],
    'ИНН': ['inn', 'EIN', 'ein', 'INN', 'tin', 'TIN'],
    'СНИЛС': ['snils', 'SNILS', 'inila', 'INILA', 'IIAN', 'iian'],
    'Отношение к воинской обязанности': ['military_service_info', 'military_status', 'mil_status', 'military_s', 'military_service_status'],
    'Сведения о трудовом стаже, предыдущих местах работы': ['work_experience', 'previous_jobs', 'employment_history', 'job_experience', 'work_years', 'job_years', 'employment_years'],
    'Семейное положение': ['marital_status', 'family_status', 'marital', 'marriage', 'fam_status'],
    'Данные об образовании, квалификации': ['education_info', 'qualification_info', 'degree_info', 'education', 'qualification', 'degree']
}
mappings_label = {
    'Полное имя': 1,
    'Пол': 2,
    'Дата рождения': 3,
    'Паспорт': 4,
    'Место и адрес жительства': 5,
    'Номер телефона': 6,
    'Сведения о доходах': 7,
    'ИНН': 8,
    'СНИЛС': 9,
    'Отношение к воинской обязанности': 10,
    'Сведения о трудовом стаже, предыдущих местах работы': 11,
    'Семейное положение': 12,
    'Данные об образовании, квалификации': 13,
    'Не является ПДн': 14
}
mappings_types = {
    'Полное имя': ['char', 'varchar', 'text'],
    'Пол': ['char', 'varchar', 'text'],
    'Дата рождения': ['char', 'varchar', 'text', 'date', 'time', 'datetime', 'timestamp'],
    'Паспорт': ['char', 'varchar', 'text'],
    'Место и адрес жительства': ['char', 'varchar', 'text'],
    'Номер телефона': ['char', 'varchar', 'text'], # числовые? но только те, что без ' ', '-', '()'
    'Сведения о доходах': ['char', 'varchar', 'text', 'integer', 'numeric', 'decimal'], # float? double?
    'ИНН': ['char', 'varchar', 'text', 'integer', 'numeric', 'decimal'],
    'СНИЛС': ['char', 'varchar', 'text'],
    'Отношение к воинской обязанности': ['char', 'varchar', 'text'],
    'Сведения о трудовом стаже, предыдущих местах работы': ['char', 'varchar', 'text'],
    'Семейное положение': ['char', 'varchar', 'text'],
    'Данные об образовании, квалификации': ['char', 'varchar', 'text'],
    'Не является ПДн': ['char', 'varchar', 'text']
}

In [50]:
tokenizer = BertTokenizer.from_pretrained(
    'google-bert/bert-base-uncased', 
    return_attention_mask=False,
    return_token_type_ids=False
)
model_bert = BertModel.from_pretrained('google-bert/bert-base-uncased')

In [57]:
def embed_bert_cls(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()

In [66]:
def one_hot_encode(text, num_classes=15):
    one_hot = to_categorical(text, num_classes=num_classes)
    return one_hot

In [79]:
model = Model()
model.load('models\\128_nadam_2drop.keras')

In [375]:
eval_test_df = pd.DataFrame(columns=['type', 'data'])

vals = {
    'Даничкин Антон Сергеевич': 'char',
    'Безопасность Безопасность Безопасность': 'text',

    'Муж': 'char',
    'Х': 'text',

    '01-01-1990': 'char',
    '12 56 78': 'text',

    '72 20 567123': 'char',
    '73 40593 848583468': 'text',

    'ЯНАО, г. Саеехард, ул. Кошшшшкино, д. 500': 'varchar',
    'Где-то там, 620, может быть и не там, да и не совсем уверен': 'text',

    '+7 922 28 60 923': 'char',
    '2 147 483 647': 'text',

    '70000': 'char',
    '250000000000000': 'text',

    '963538512189': 'char',
    '123123456456': 'text',

    '146-533-459 14': 'char',
    '123-123-123-123': 'text',

    'Годен с ограничениями': 'char',
    'Это никуда не годится': 'text',

    'Блогер': 'char',
    'Размышления и полет фантазии': 'text',

    'Не замужем': 'char',
    'Под окном': 'text',

    'Незаконченное высшее': 'char',
    'Два маленьких, два больших и два средних': 'text',   
}

for key, value in vals.items():
    eval_test_df = eval_test_df.append({
        'data': key,
        'type': value,
    }, ignore_index=True)

In [376]:
eval_test_df = pd.DataFrame(columns=['type', 'data'])

vals = {

    'Незаконченное высшее': 'char',
    'Два маленьких, два больших и два средних': 'text',
}

for key, value in vals.items():
    eval_test_df = eval_test_df.append({
        'data': key,
        'type': value,
    }, ignore_index=True)

In [377]:
eval_test_df

,type,data
0,char,Даничкин Антон Сергеевич
1,text,Безопасность Безопасность Безопасность
2,char,Муж
3,text,Х
4,char,01-01-1990
5,text,12 56 78
6,char,72 20 567123
7,text,73 40593 848583468
8,varchar,"ЯНАО, г. Саеехард, ул. Кошшшшкино, д. 500"
9,text,"Где-то там, 620, может быть и не там, да и не ..."


In [378]:
eval_test_df_original = eval_test_df.copy()

In [379]:
eval_test_df['type'] = lb.fit_transform(eval_test_df['type'])
eval_test_df['type'] = eval_test_df['type'].apply(one_hot_encode)
eval_test_df['data'] = eval_test_df['data'].apply(lambda x: embed_bert_cls(x, model_bert, tokenizer))

type_eval = np.stack(eval_test_df['type'].values)
data_eval = np.stack(eval_test_df['data'].values)

type_eval = np.zeros((type_eval.shape[0], 255), dtype=np.float32)
type_eval[:, :type_eval.shape[1]] = type_eval

In [380]:
predictions = model.predict(
    {
        'i_type': type_eval, 
        'i_data': data_eval
    })

1/1 [==============================] - 0s 135ms/step


In [381]:
predictions_list = predictions.tolist()

In [393]:
predictions_df = pd.DataFrame(columns=['type', 'data', 'prediction'])

for i in range(len(predictions_list)):
    col1, col2 = eval_test_df_original['type'][i], eval_test_df_original['data'][i]
    sorted_indices = sorted(range(len(predictions_list[i])), key=lambda k: predictions_list[i][k], reverse=True)
    filtered_indices = [idx for idx in sorted_indices if predictions_list[i][idx] >= 0.3]
    predictions_str = ', '.join([f'{list(mappings_label.keys())[list(mappings_label.values()).index(idx+1)]} - {predictions_list[i][idx]:.4f}' for idx in filtered_indices])
    predictions_df = predictions_df.append({
        'type': col1,
        'data': col2,
        'prediction': predictions_str
    }, ignore_index=True)

predictions_df

,type,data,prediction
0,char,Даничкин Антон Сергеевич,Полное имя - 0.9959
1,text,Безопасность Безопасность Безопасность,Не является ПДн - 0.9998
2,char,Муж,Пол - 0.9967
3,text,Х,Не является ПДн - 0.9992
4,char,01-01-1990,Дата рождения - 0.9666
5,text,12 56 78,Не является ПДн - 0.8857
6,char,72 20 567123,Паспорт - 0.9481
7,text,73 40593 848583468,Не является ПДн - 0.9740
8,varchar,"ЯНАО, г. Саеехард, ул. Кошшшшкино, д. 500",Место и адрес жительства - 0.9997
9,text,"Где-то там, 620, может быть и не там, да и не ...",Не является ПДн - 0.9998
